### Camera access information



In [ ]:
username ="admin"
password = "nasr1234"
cctv_ip_1 = "192.168.0.151"
cctv_ip_2 = "192.168.0.152"

cctv_1_url = f'rtsp://{username}:{password}@{cctv_ip_1}/cam/realmonitor?channel=1&subtype=00&authbasic=YWRtaW46YWRtaW4='
cctv_2_url = f'rtsp://{username}:{password}@{cctv_ip_2}/cam/realmonitor?channel=1&subtype=00&authbasic=YWRtaW46YWRtaW4='


### Import required modules

In [ ]:
import cv2
import stitching
import numpy as np
from pathlib import Path
import os
import glob
from matplotlib import pyplot as plt
import imageio

### Define helper functions 
- Function 1 & 2: to plot images
- Function 3: to resize frames
- Function 3: to save videos

In [ ]:
def plot_image(img, figsize_in_inches=(5,5)):
    fig, ax = plt.subplots(figsize=figsize_in_inches)
    ax.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.show()

def plot_images(imgs, figsize_in_inches=(5,5)):
    fig, axs = plt.subplots(1, len(imgs), figsize=figsize_in_inches)
    for col, img in enumerate(imgs):
        axs[col].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.show(block=False)

def resize_image(img, percent = 50):
    scale_percent = percent
    width = int(img.shape[1] * scale_percent / 100)
    height = int(img.shape[0] * scale_percent / 100)
    dim = (width, height)
    resized_img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
    return resized_img

def write_video(cap):
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    fps = cap.get(cv2.CAP_PROP_FPS)

    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    out = cv2.VideoWriter('out_camera.avi', fourcc, fps, (width, height))
    return out

# Sometimes, and due to codec related issues, cv2.videoWriter metod does not work. In that case, use imageio
def write_video_imageio(cap):
    writer = imageio.get_writer('tracking.avi', fps=cap.get(cv2.CAP_PROP_FPS))
    return writer



### Caibrate the camera

In [ ]:
def calibrate_camera(board_size, square_size, calib_file_dest_path, calibration_images):
    
    #used for calibration
    board_size = (11, 7)
    square_size = 70  # Length of one side of a square in mm

    calib_file_dest_path = "./Calibration_data/calibration_data_Mob_MRS.npz"
    obj_points = []  # 3D points in real world space
    img_points = []  # 2D points in image plane

    objp = np.zeros((board_size[0] * board_size[1], 3), np.float32)
    objp[:, :2] = np.mgrid[0:board_size[0], 0:board_size[1]].T.reshape(-1, 2) * square_size

    calibration_images = glob.glob('./Calibration_data/MOB_Images/*.png')

    for frame in calibration_images:
        img = cv2.imread(frame)
        resized_img = resize_image(img, percent = 30)
        gray = cv2.cvtColor(resized_img, cv2.COLOR_BGR2GRAY)

        ret, corners = cv2.findChessboardCorners(gray, board_size, None)

        if ret:
            obj_points.append(objp)

            corners2 = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001))
            img_points.append(corners2)

            img = cv2.drawChessboardCorners(resized_img, board_size, corners2, ret)
            cv2.imshow('Checkerboard', resized_img)
            cv2.waitKey(10)
    
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(obj_points, img_points, gray.shape[::-1], None, None)

    # Save the calibration data as an npz file
    np.savez(calib_file_dest_path, mtx=mtx, dist=dist)

    cv2.destroyAllWindows()

### Load Calibration data

In [ ]:
calibration_file_path = './Calibration_data/calibration_data_cctv_1.npz'
# calibration_file_path = './Calibration_data/calibration_data_webcam_1.npz'

with np.load(calibration_file_path) as data:

    cam_mtx, dist_coeff = data['mtx'], data['dist']

#For mobile camera, we can assume there is no distortion
zero_dist_coeff = np.array([[0.0, 0.0, 0.0, 0.0, 0.0]])

### Define three functions:
 - Function 1: undistort the frame
 - Function 2: create the auco marker detector
 - Function 3: stitch the images

In [ ]:
def undistort_frame(frame, mtx, dist):
    h, w = frame.shape[:2]

    # mtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, (w, h), 0, (w, h))

    undistorted_frame = cv2.undistort(frame, mtx, dist)

    #For most cases, this step doesn't produce better results
    #x, y, w, h = roi
    #undistorted_frame = undistorted_frame[y:y+h, x:x+w]

    return undistorted_frame

def create_detector(dictionary = cv2.aruco.DICT_5X5_250):
    aruco_dict = cv2.aruco.getPredefinedDictionary(dictionary)
    params = cv2.aruco.DetectorParameters()
    return cv2.aruco.ArucoDetector(aruco_dict, params)

#used to stitch unsaved frames, since the stitcher class only accepts file paths
def stitch(stitcher, frames):

    temp_files = []
    for i, frame in enumerate(frames):
        temp_file = f"temp_{i}.jpg"
        cv2.imwrite(temp_file, frame)
        temp_files.append(temp_file)

    stitched_image = stitcher.stitch([temp_files[0], temp_files[1]])

    for temp_file in temp_files:
        os.remove(temp_file)
    
    return stitched_image

###  Perform projective transformortaion if needed

In [ ]:
# Initialize global variables
src_points = []
dst_points = []

def mouse_click(event, x, y, flags, param):
    # Handle mouse click events
    if event == cv2.EVENT_LBUTTONDOWN:
        if len(src_points) < 4:
            # Add the clicked point to the source points list
            src_points.append((x, y))
            cv2.circle(input_image, (x, y), 5, (0, 0, 255), -1)
            cv2.imshow("Input Image", input_image)


# Load the input image
input_image = cv2.imread('./hellos_KF_2.png')
# input_image = resize_image(image, percent = 30)
height, width = input_image.shape[:2]

# Create windows and set mouse click callback
cv2.namedWindow("Input Image", cv2.WINDOW_NORMAL)
# cv2.namedWindow("Transformed Image", cv2.WINDOW_NORMAL)

cv2.setMouseCallback("Input Image", mouse_click)

while True:
    cv2.imshow("Input Image", input_image)
    key = cv2.waitKey(1) & 0xFF

    if key == ord("r"):
        src_points = []
        dst_points = []
        # input_image = resize_image(image, percent = 30)
        cv2.destroyWindow("Input Image")
        input_image = cv2.imread('./hellos_KF_2.png')
        cv2.namedWindow("Input Image", cv2.WINDOW_NORMAL)
        cv2.setMouseCallback("Input Image", mouse_click)
        cv2.imshow("Input Image", input_image)
        if cv2.getWindowProperty("Transformed Image", cv2.WND_PROP_VISIBLE) >= 1:
            cv2.destroyWindow("Transformed Image")

    elif key == ord("c"):
        width, height = input_image.shape[:2]
        dst_points.append((0, 0))
        dst_points.append((width, 0))
        dst_points.append((width, height))
        dst_points.append((0, height))
        if len(src_points) == 4:
            # Calculate and display the perspective transformation
            M = cv2.getPerspectiveTransform(np.float32(src_points), np.float32(dst_points))
            transformed_image = cv2.warpPerspective(input_image, M, (width, height))
            cv2.imshow("Transformed Image", transformed_image)

    elif key == ord('q'):
        # cv2.imwrite("transformed_image_right.jpg", transformed_image)
        break

cv2.destroyAllWindows()


### Perform stitching

In [ ]:
# initialize stitcher
settings = {"detector": "sift", "confidence_threshold": 0.1}
stitcher = stitching.Stitcher(**settings)

# read the images for stitching
city_imgs = []  
for path in Path('./Mobile_Dataset/New folder/first').glob("*"):
    city_imgs.append(str(path))

# stitch the images
stitched_image = stitcher.stitch(stitcher, city_imgs)

## These are optional steps to detect keypoints and matches between images and draw them if needed

img_handler = stitching.image_handler.ImageHandler(medium_megapix=0.6, low_megapix=0.1, final_megapix=-1)
img_handler.set_img_names(city_imgs)

# resize images
medium_imgs = list(img_handler.resize_to_medium_resolution())
low_imgs = list(img_handler.resize_to_low_resolution(medium_imgs))
final_imgs = list(img_handler.resize_to_final_resolution())
# plot_images(list(final_imgs), (20,20))

#draw detected keypoints
finder = stitching.feature_detector.FeatureDetector()
features = [finder.detect_features(img) for img in medium_imgs]
keypoints_center_img_1 = finder.draw_keypoints(medium_imgs[0], features[0])
keypoints_center_img_2 = finder.draw_keypoints(medium_imgs[1], features[1])
keypoints_center_img_3 = finder.draw_keypoints(medium_imgs[1], features[1])
# plot_images([keypoints_center_img_1,keypoints_center_img_2, keypoints_center_img_3])

#plot matched features
matcher = stitching.feature_matcher.FeatureMatcher()
matches = matcher.match_features(features)
print(matches)
print(matcher.get_confidence_matrix(matches))

all_relevant_matches = matcher.draw_matches_matrix(final_imgs, features, matches, conf_thresh=0.01, 
                                                    inliers=True, matchColor=(0, 255, 0))
print(all_relevant_matches)
for idx1, idx2, img in all_relevant_matches:
    print(idx1, idx2)
    print(f"Matches Image {idx1+1} to Image {idx2+1}")
    plot_image(img, (200,300))
    break

### Detect the aruco markers and calculate their actual and relative coordinates

In [ ]:
# ArUco marker ID from which positions orientations of
#    all other aruco markers are measured.
world_id = 10

# Size of the ArUco marker in mm
marker_size = 200

"""
The correction factor is used to correct the position
    of the ArUco marker in the distortion axis of the camera.
It's estimated by trial and error
"""
correction_factor = 1.2/1.35

# initialize the ArUco detector
detector = create_detector()

#capture the frame
# cap = cv2.VideoCapture('help.avi')
#or
#read the image
undistorted_frame = cv2.imread("./Webcam1275.jpg")   

# writer = write_video(cap)
# or
# writer = write_video_imageio(cap)

while True:

    # ret, undistorted_frame = cap.read()
    # if not ret:
    #     break

    # resize the image in case it's too large
    # resized_img = resize_image(frame, percent = 75) 

    # This step does not make a difference in the results
    # original_gray = cv2.cvtColor(resized_img, cv2.COLOR_BGR2GRAY)

    # undistort the frame
    # undistorted_frame = undistort_frame(resized_img, cam_mtx, dist_coeff)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    # In case the image is not bright enough for ArUco detection, we can use a kernel to increase the brightness
    # kernel = np.array([[-1,-1,-1], [-1,11,-1], [-1,-1,-1]])    
    # undistorted_frame = cv2.filter2D(resized_img, -1, kernel)

    # detect the ArUco markers
    corners, ids, _ = detector.detectMarkers(undistorted_frame)

    if ids is None:
    #   cv2.rectangle(resized_img, (0, 0), (resized_img.shape[1], resized_img.shape[0]), (0, 0, 255), thickness=2)
    #   cv2.putText(resized_img,"No aruco markers detected", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    #   cv2.imshow('empty_frame', undistorted_frame)
        print("No aruco markers detected")
        continue
 

    if cv2.getWindowProperty('empty_frame', cv2.WND_PROP_VISIBLE) == 1:
        cv2.destroyWindow('empty_frame')

    # draw boxes around the detected ArUco markers
    cv2.aruco.drawDetectedMarkers(undistorted_frame, corners, ids)

    # estimate the pose of the ArUco markers
    rvecs, tvecs, _ = cv2.aruco.estimatePoseSingleMarkers(corners, marker_size, cam_mtx, zero_dist_coeff)

    for i, id in enumerate(ids.flatten()):
        if id == world_id:
            world_pos_vec = tvecs[i].squeeze()
            world_rot_vec = rvecs[i].squeeze()

            # For debugging purposes
            print("################################################## \n")
            print(f"position vector of world aruco marker with {id} = {world_pos_vec[:2]/1000} \n")
            print(f"rotation matrix of world aruco marker with {id} = {world_rot_vec} \n")

            # draw the coordinate axes of the world ArUco marker
            cv2.drawFrameAxes(undistorted_frame, cam_mtx, zero_dist_coeff, world_rot_vec, world_pos_vec, marker_size) 
            cv2.putText(undistorted_frame, "World", (int(corners[i][0][0][0] + 10), int(corners[i][0][0][1] + 45)), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 0), 2)                   
        else:
            world_idx = np.argmax(ids == world_id)

            # The translation and rotation vectors of the world ArUco marker
            world_pos_vec = tvecs[world_idx].squeeze()
            world_rot_vec = rvecs[world_idx].squeeze()


            # The translation and rotation vectors of the other ArUco marker
            aruco_pos_vec = tvecs[i].squeeze()
            aruco_rot_vec = rvecs[i].squeeze()

            # convert the rotation vector to a rotation matrix
            world_rot_mtx, _ = cv2.Rodrigues(world_rot_vec)
            aruco_rot_mtx, _ = cv2.Rodrigues(aruco_rot_vec)
            
            # calculate the translation vector and rotation vector of the other ArUco marker relative to the world ArUco marker
            aruco_rel_pos_vec = world_rot_mtx.T @ (aruco_pos_vec - world_pos_vec)
            aruco_rel_rot_mtx = aruco_rot_mtx @ world_rot_mtx.T

            # extract the yaw angle of the other ArUco marker relative to the world ArUco marker
            yaw_angle_rad = np.arctan2(aruco_rel_rot_mtx[0, 1], aruco_rel_rot_mtx[0, 0])

            # convert the yaw angle from radians to degrees
            yaw_angle_deg = np.rad2deg(yaw_angle_rad)

            # correct the position of the other ArUco marker in the distortion axis of the cctv camera
            #    which in this case is the y-axis
            aruco_rel_pos_vec[1] *= correction_factor

            # For debugging purposes
            print("################################################## \n")
            print(f"Relative position vector for marker with id {id} ", aruco_rel_pos_vec[:2]/1000)
            print(f"The yaw angle for marker with id {id}:", yaw_angle_deg)

            # draw the coordinate axes of the ArUco marker
            cv2.drawFrameAxes(undistorted_frame, cam_mtx, zero_dist_coeff, aruco_rot_vec, aruco_pos_vec, marker_size) 


            # draw the information of each ArUco marker
            info_1 = f"Position: {round(aruco_rel_pos_vec[0]/1000, 3)}, {round(aruco_rel_pos_vec[1]/1000, 3)} m"
            info_2 = "Orientation :" + str(round(yaw_angle_deg, 3)) + " deg"
            cv2.putText(undistorted_frame, info_1, (int(corners[i][0][0][0] - 120), int(corners[i][0][0][1] - 40)), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 2)
            cv2.putText(undistorted_frame, info_2, (int(corners[i][0][0][0] - 120), int(corners[i][0][0][1]) -20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 2)
        
    if undistorted_frame is not None:
        cv2.imshow('frame', undistorted_frame)
        cv2.imwrite("./R_555_webcam", undistorted_frame)
        # to save a video 
        # writer.write(undistorted_frame)
        # or
        # writer.append_data(cv2.cvtColor(undistorted_frame, cv2.COLOR_BGR2RGB))

        # to save a frame
        # cv2.imwrite(f'./estimate_pose/{idx}.jpg', undistorted_frame)

# cap.release()
# writer.close()
cv2.destroyAllWindows()
